In [ ]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt 

## Load Communes, Competitions and Shape data

In [ ]:
# Change coordiantes
df_communes = pd.read_csv("..\Data\\Communes\\communes.csv", sep=";")

df_communes = df_communes.drop_duplicates(subset="Commune", keep="first")

gdf = gpd.GeoDataFrame(
    df_communes,
    geometry=[Point(e, n) for e, n in zip(df_communes.E, df_communes.N)],
    crs="EPSG:2056"  # LV95
)

gdf = gdf.to_crs(epsg=4326)

df_communes["lat"] = gdf.geometry.y
df_communes["lon"] = gdf.geometry.x

df_communes.to_csv("..\Data\\Communes\\communes.csv", sep=";")


In [ ]:
with open('..\Data\geo_data\canton_name_to_abbreviation.json', encoding='utf-8') as f:
    canton_to_abbreviation = json.load(f)

gdf_communes = gpd.read_file("..\Data\\geo_data\\swiss_shp\\swissBOUNDARIES3D_1_5_TLM_HOHEITSGEBIET.shp") 
gdf_communes = gdf_communes.drop(["DATUM_AEND", "DATUM_ERST"], axis=1)
gdf_communes = gdf_communes.rename(columns={"NAME": "Commune"})
gdf_communes = gdf_communes.to_crs(epsg=4326)


gdf_cantons = gpd.read_file("..\Data\\geo_data\\swiss_shp\\swissBOUNDARIES3D_1_5_TLM_KANTONSGEBIET.shp")
gdf_cantons = gdf_cantons.drop(["DATUM_AEND", "DATUM_ERST"], axis=1)
gdf_cantons = gdf_cantons.rename(columns={"NAME": "Canton"})
gdf_cantons = gdf_cantons.to_crs(epsg=4326)
gdf_cantons["Canton"] = gdf_cantons["Canton"].replace(canton_to_abbreviation)

In [ ]:
gdf_communes[gdf_communes["Commune"] == "Leytron"]

In [ ]:
df_competitions = pd.read_feather("..\Data\\ACM\\AcmEPFL_paired_communes.feather")  # Adjust name

df = df_competitions.merge(df_communes, on="Commune", how="left")
df = df.dropna(subset=["lat", "lon"])

## Map of Competitions

In [ ]:
m = folium.Map(location=[46.8182, 8.2275], tiles="Cartodb Positron", zoom_start=8)
marker_cluster = MarkerCluster().add_to(m)


for _, row in df.iterrows():
    folium.Marker(
        location=[row["lat"], row["lon"]],
        popup=f"{row["Nom de l'objet"]}",
        tooltip=row["Commune"]
    ).add_to(marker_cluster)

""" # Step 8: Save or display map
m.save("swiss_competitions_map.html") """
m


## Map of Competitons per Canton

In [ ]:
#Group competions by canton

comp_counts = df_competitions["Canton"].value_counts().reset_index()
comp_counts.columns = ["Canton", "count"]

gdf_cantons = gdf_cantons.merge(comp_counts, on="Canton", how="left")
gdf_cantons["count"] = gdf_cantons["count"].fillna(0)
gdf_cantons = gdf_cantons.drop(columns=["count_x", "count_y"], errors='ignore')

In [ ]:
threshold_scale = [0, 1, 50, 100, 150, 200, 250, 308] 

In [ ]:
m = folium.Map(location=[46.8, 8.3], tiles="Cartodb Positron", zoom_start=8)


folium.Choropleth(
    geo_data=gdf_cantons.to_json(),  
    data=gdf_cantons,
    columns=["Canton", "count"],
    key_on="feature.properties.Canton",  
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    threshold_scale=threshold_scale,
    legend_name="Number of Competitions per Canton"
).add_to(m)

folium.GeoJson(
    gdf_cantons,
    name="Canton Info",
    tooltip=folium.GeoJsonTooltip(
        fields=["Canton", "count"],
        aliases=["Canton:", "Concours:"],
        localize=True
    ),
    style_function=lambda x: {
        "color": "transparent",    
        "weight": 0,               
        "fillOpacity": 0           
    },
).add_to(m)

folium.LayerControl().add_to(m)
m

In [ ]:
m = folium.Map(location=[46.8, 8.3], tiles="Cartodb Positron", zoom_start=8)


folium.Choropleth(
    geo_data=gdf_cantons.to_json(),  
    name="choropleth",
    data=gdf_cantons,
    columns=["Canton", "count"],
    key_on="feature.properties.Canton",  
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    threshold_scale=threshold_scale,
    legend_name="Number of Competitions per Canton"
).add_to(m)

folium.GeoJson(
    gdf_cantons,
    name="Canton Info",
    tooltip=folium.GeoJsonTooltip(
        fields=["Canton", "count"],
        aliases=["Canton:", "Concours:"],
        localize=True
    ),
    style_function=lambda x: {
        "color": "transparent",    
        "weight": 0,               
        "fillOpacity": 0           
    },
).add_to(m)

folium.LayerControl().add_to(m)
m

## Map of Competitons per Commune

In [ ]:
#Group competions by canton

comp_counts = df_competitions["Commune"].value_counts().reset_index()
comp_counts.columns = ["Commune", "count"]
# Assume gdf_cantons contains geometries at commune level and has a 'Canton' column
gdf_communes_merged = gdf_communes.merge(comp_counts, on="Commune", how="left")
gdf_communes_merged["count"] = gdf_communes_merged["count"].fillna(0)
gdf_communes_merged = gdf_communes_merged.drop(columns=["count_x", "count_y"], errors='ignore')

In [ ]:
threshold_scale = [0, 1, 5, 10, 20, 40, 80, 130]  # Must include the maximum count

In [ ]:
m = folium.Map(location=[46.8, 8.3], tiles="Cartodb Positron", zoom_start=8)


folium.Choropleth(
    geo_data=gdf_communes_merged.to_json(), 
    name="choropleth",
    data=gdf_communes_merged,
    columns=["Commune", "count"],
    key_on="feature.properties.Commune",  
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    threshold_scale=threshold_scale,
    legend_name="Nombre de concours par commune"
).add_to(m)

folium.GeoJson(
    gdf_communes_merged,
    name="Commune Info",
    tooltip=folium.GeoJsonTooltip(
        fields=["Commune", "count"],
        aliases=["Commune:", "Concours:"],
        localize=True
    ),
    style_function=lambda x: {
        "color": "transparent",    
        "weight": 0,               
        "fillOpacity": 0           
    },
).add_to(m)

folium.LayerControl().add_to(m)
m

## Commune map, removes all communes with no appearences

In [ ]:
gdf_communes_appearences_only = gdf_communes_merged[gdf_communes_merged["count"] > 0]

In [ ]:
m = folium.Map(location=[46.8, 8.3], tiles="Cartodb Positron", zoom_start=8)


folium.Choropleth(
    geo_data=gdf_communes_appearences_only.to_json(),  
    name="choropleth",
    data=gdf_communes_appearences_only,
    columns=["Commune", "count"],
    key_on="feature.properties.Commune",  
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    threshold_scale=threshold_scale,
    legend_name="Number of Competitions per Commune"
).add_to(m)

folium.GeoJson(
    gdf_communes_appearences_only,
    name="Commune Info",
    tooltip=folium.GeoJsonTooltip(
        fields=["Commune", "count"],
        aliases=["Commune:", "Concours:"],
        localize=True
    ),
    style_function=lambda x: {
        "color": "transparent",    
        "weight": 0,               
        "fillOpacity": 0           
    },
).add_to(m)

folium.LayerControl().add_to(m)
m

In [ ]:
df_competitions = df_competitions.sort_values(by="Date de début de l'objet")

df_competitions_cumulative = df_competitions[["Date de début de l'objet",'Canton']]
df_competitions_cumulative["Date de début de l'objet"] = pd.DatetimeIndex(df_competitions_cumulative["Date de début de l'objet"]).year
df_competitions_cumulative["Count"] = 1
comptitions_grouped = df_competitions_cumulative.groupby(['Date de début de l\'objet', 'Canton'])['Count'].sum().reset_index()
comptitions_grouped

In [ ]:
grouped = comptitions_grouped.sort_values('Date de début de l\'objet')

pivot = grouped.pivot(index='Date de début de l\'objet', columns='Canton', values='Count').fillna(0)
cumulative = pivot.cumsum()

#Specifiy which cantons:
cantons = ["VD", "VS", "BE", "GE", "NE", "BE", "FR"]

cumulative = cumulative[cantons]

cumulative.plot.area(figsize=(14, 8), colormap='tab20')

plt.title("Cumulative Competitions by Canton Over Time")
plt.xlabel("Date")
plt.ylabel("Cumulative Number of Competitions")
plt.legend(title="Canton")
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:
df_competitions['Canton'].value_counts()